# TensorRT

Dalam notebook ini, kita akan menggunakan TensorRT untuk mengoptimalkan model PyTorch untuk inferensi. Kita akan melatih model CNN sederhana pada set data MNIST, mengonversinya ke mesin TensorRT menggunakan ONNX, lalu melakukan inferensi menggunakan model mesin TensorRT yang dioptimalkan dan mengevaluasi ukuran dan akurasi model. Notebook ini memerlukan GPU NVIDIA dengan dukungan CUDA atau perangkat NVIDIA Jetson.

## Siapkan TensorRT

Pertama, instal tensorrt dan torch menggunakan pip dan impor modul yang diperlukan

In [1]:
%pip install torch torchvision
%pip install tensorrt==8.6.1
%pip install pycuda
%pip install pycuda onnx onnxruntime
%pip install onnxruntime
%pip install --no-cache-dir --extra-index-url https://pypi.nvidia.com pytorch-quantization==2.1.2

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1-py2.py3-none-any.whl size=16972 sha256=6912c4bfcd223683c8b77af08b1c02425e41db7ac35155b7070bf943b05e8521
  Stored in directory: /root/.cache/pip/wheels/6d/29/56/abdffd4c604f255b5254bef3f1c598ab7811ea020540599438
Successfully built tensorrt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.4/92.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=62998700d6bf33499e3cca1b9d1202c6bf28261dcf7bb163270852ad148286cb
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycu

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.quantization
import pathlib
import numpy as np
import torch.onnx
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import onnx
import onnxruntime

from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization import calib
from tqdm import tqdm

## Latih Model PyTorch dan Ekspor ke ONNX

Selanjutnya, latih model CNN sederhana pada dataset MNIST dan ekspor ke format ONNX

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_dataset = datasets.MNIST('./data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST('./data', train=False,transform=transform)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(12 * 13 * 13, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output


train_loader = torch.utils.data.DataLoader(train_dataset, 32)
test_loader = torch.utils.data.DataLoader(test_dataset, 32)

device = "cpu"

epochs = 1

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()

for epoch in range(1, epochs+1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

MODEL_DIR = pathlib.Path("./models")
MODEL_DIR.mkdir(exist_ok=True)
torch.save(model.state_dict(), MODEL_DIR / "original_model.p")

x, _ = next(iter(train_loader))
torch.onnx.export(model,
                  x,
                  MODEL_DIR / "mnist_model.onnx",
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes={'input' : {0 : 'batch_size'},
                                'output' : {0 : 'batch_size'}})

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 513kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.52MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 9.81MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.266016
Train Epoch: 1 [32/60000 (0%)]	Loss: 2.224627
Train Epoch: 1 [64/60000 (0%)]	Loss: 1.996070
Train Epoch: 1 [96/60000 (0%)]	Loss: 1.784491
Train Epoch: 1 [128/60000 (0%)]	Loss: 1.953514
Train Epoch: 1 [160/60000 (0%)]	Loss: 1.990250
Train Epoch: 1 [192/60000 (0%)]	Loss: 1.399367
Train Epoch: 1 [224/60000 (0%)]	Loss: 1.552165
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.584013
Train Epoch: 1 [288/60000 (0%)]	Loss: 1.240830
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.287110
Train Epoch: 1 [352/60000 (1%)]	Loss: 1.143005
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.111068
Train Epoch: 1 [416/60000 (1%)]	Loss: 1.070271
Train Epoch: 1 [448/60000 (1%)]	Loss: 0.995884
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.435712
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.175349
Train Epoch: 1 [544/60000 (1%)]	Loss: 0.924379
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.143811
Train Epoch: 1 [608/60000 (1%)]	Loss:

## Mengonversi Model ONNX ke TensorRT

Untuk mengonversi model ONNX ke mesin TensorRT menggunakan API Python TensorRT. Pertama, inisialisasi komponen TensorRT yang merupakan logger, builder, dan jaringan. Selanjutnya, tentukan parser ONNX untuk mengurai model ONNX dari file ONNX ke jaringan TensorRT. Kemudian, buat konfigurasi builder untuk menetapkan parameter pembangunan dan batas kumpulan memori untuk ruang kerja di TensorRT. Kemudian, buat profil pengoptimalan untuk menangani bentuk masukan dinamis dengan ukuran batch 32, ukuran saluran 1, dan dimensi gambar 28x28. Selanjutnya, bangun dan serialkan mesin TensorRT menggunakan jaringan dan builder yang dikonfigurasi, lalu simpan ke disk. Terakhir, skrip dibersihkan dengan menghapus objek builder dan jaringan untuk mengosongkan sumber daya.

In [4]:
onnx_path = MODEL_DIR / "mnist_model.onnx"
trt_path = MODEL_DIR / 'mnist_engine_pytorch.trt'

# inisialisasi engine TensorRT dan parsing model ONNX
logger = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(logger)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

parser = trt.OnnxParser(network, logger)
parser.parse_from_file(str(onnx_path))

# menyiapkan konfigurasi builder dan profil optimasi
config = builder.create_builder_config()
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)

profile = builder.create_optimization_profile()
profile.set_shape("input", (32, 1, 28, 28), (32, 1, 28, 28), (32, 1, 28, 28))
config.add_optimization_profile(profile)

# serialisasi engine, kemudian simpan ke disk
serialized_engine = builder.build_serialized_network(network, config)
with open(str(trt_path), 'wb') as f:
    f.write(serialized_engine)

# bebaskan sumber daya
del builder
del network


## Jalankan Inferensi dan Periksa Akurasi

Terakhir, jalankan inferensi lalu bandingkan akurasi model engine TensorRT dengan model ONNX pada set data pengujian.

Untuk menjalankan model ONNX pengujian, muat model dan integritas model pengujian lalu ulangi Data Loader yang diberikan. Untuk setiap batch, ubah data input ke array NumPy dan masukkan ke sesi ONNX Runtime. Setelah memperoleh output, ubah kembali ke tensor PyTorch. Lalu, hitung akumulasi kerugian log likelihood negatif
dan jumlah prediksi yang benar untuk mengukur akurasi model.

Untuk menguji model tensorRT, pertama-tama, muat engine serial dari disk, dan inisialisasi runtime TensorRT. Kemudian, deserialisasi engine dan buat konteks eksekusi. Selanjutnya, alokasikan memori untuk data input dan output pada GPU, atur binding untuk eksekusi TensorRT, dan buat aliran CUDA untuk mengelola transfer data asinkron antara CPU dan GPU. Kemudian, Ulangi Data Loader yang diberikan dan untuk setiap batch, ubah data input menjadi array NumPy dan transfer ke GPU, sebelum menjalankan model secara asinkron, lalu transfer prediksi kembali ke CPU. Jalankan sinkronisasi untuk memastikan koordinasi yang tepat antara thread. Selanjutnya, bentuk ulang output dan ubah ke tensor PyTorch untuk menghitung akumulasi kerugian log likelihood negatif dan jumlah prediksi yang benar untuk mengukur akurasi model. Terakhir, kosongkan memori dan sumber daya CUDA

In [5]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def test_onnx(model_name, data_loader):
    onnx_model = onnx.load(model_name)
    onnx.checker.check_model(onnx_model)
    ort_session = onnxruntime.InferenceSession(model_name)
    test_loss = 0
    correct = 0
    for data, target in data_loader:
        ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(data)}
        output = ort_session.run(None, ort_inputs)[0]
        output = torch.from_numpy(output)
        if target.shape[0] == 32: # batch terakhir mungkin lebih kecil dari 32 (perbaikan sementara)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # jumlahkan loss batch
            pred = output.argmax(dim=1, keepdim=True)  # ambil indeks dengan probabilitas log maksimum
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(data_loader.dataset)
    return 100. * correct / len(data_loader.dataset)

def test_tensorrt(model_name, data_loader):
    with open(model_name, "rb") as f:
        serialized_engine = f.read()
    runtime = trt.Runtime(logger)
    engine = runtime.deserialize_cuda_engine(serialized_engine)
    context = engine.create_execution_context()
    input_size = trt.volume(engine.get_binding_shape(0))
    output_size = trt.volume(engine.get_binding_shape(1))
    # Alokasikan memori di perangkat
    d_input = cuda.mem_alloc(input_size * 4)  # Asumsikan tipe data float32 4-byte
    d_output = cuda.mem_alloc(output_size * 4)
    bindings=[int(d_input), int(d_output)]
    stream = cuda.Stream()
    h_output = np.empty(output_size, dtype=np.float32)
    test_loss = 0
    correct = 0
    for data, target in data_loader:
        # Buat array numpy untuk menampung data input dan output
        h_input = data.numpy().astype(np.float32)
        # Transfer data input ke perangkat
        cuda.memcpy_htod_async(d_input, h_input, stream)
        # Eksekusi model
        context.execute_async_v2(bindings, stream.handle, None)
        # Transfer prediksi kembali
        cuda.memcpy_dtoh_async(h_output, d_output, stream)
        # Sinkronkan thread
        stream.synchronize()
        output = h_output.reshape(context.get_tensor_shape('output'))
        output = torch.from_numpy(output)
        if target.shape[0] == 32: # batch terakhir mungkin lebih kecil dari 32 (perbaikan sementara)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(data_loader.dataset)
    del context
    del engine
    cuda.Context.pop()
    return 100. * correct / len(data_loader.dataset)

acc = test_onnx(onnx_path, test_loader)
print(f"Akurasi model onnx adalah {acc}%")

trtr_acc = test_tensorrt(trt_path, test_loader)
print(f"Akurasi model tensorrt adalah {trtr_acc}%")


Akurasi model onnx adalah 96.63%


<ipython-input-5-d906667edab1>:27: DeprecationWarning: Use get_tensor_shape instead.
  input_size = trt.volume(engine.get_binding_shape(0))
<ipython-input-5-d906667edab1>:28: DeprecationWarning: Use get_tensor_shape instead.
  output_size = trt.volume(engine.get_binding_shape(1))


Akurasi model tensorrt adalah 96.63%
